In [ ]:
%load_ext autoreload
%autoreload 2

## Pre-requisites

Need to download the single batch 

```
# The following will download the data 50GB - 100GB
# to the directory:
# `~/datasets/seamless_interaction/improvised/dev/0000/`
download_single_batch()
```

to the submodule `seamless-interaction` directory to make it work.

In [ ]:
import torch
import numpy as np
from pathlib import Path
from transformers import MimiModel, AutoFeatureExtractor

In [ ]:
#
from seamless_interaction.fs import DatasetConfig, SeamlessInteractionFS
from dataset_utils.duplex import get_data_sample
from mimi.feature_extraction import get_mimi_features

In [ ]:
NUM_QUANTIZERS = 8

In [ ]:
config = DatasetConfig(label="improvised", split="dev", num_workers=8)
fs = SeamlessInteractionFS(config=config)

pair = ["V00_S0696_I00000375_P0844A", "V00_S0696_I00000375_P0847"]

In [ ]:
sample1 = get_data_sample(fs, pair[0])
sample2 = get_data_sample(fs, pair[1])

## 1. Model Initialization

In [ ]:
%%capture

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MimiModel.from_pretrained("kyutai/mimi", num_quantizers=8)
feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")

if device.type == "cuda":
    model.to(device)

## 2. Encoding and Decoding

In [ ]:
audio_arr = sample1.get_single_channel_audio(rate=feature_extractor.sampling_rate, mask_transcript=True)

In [ ]:
inputs = feature_extractor(
    raw_audio=audio_arr,
    sampling_rate=feature_extractor.sampling_rate,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

In [ ]:
encoder_outputs = model.encode(inputs["input_values"])
audio_values = model.decode(encoder_outputs.audio_codes)[0]

In [ ]:
reconstructed_audio = audio_values.data.squeeze().cpu().numpy()

In [ ]:
# Listen to the original and reconstructed audio

# print('Reconstructed audio:')
# display(Audio(reconstructed_audio, rate=feature_extractor.sampling_rate))
# print('Original audio:')
# display(Audio(audio_arr, rate=feature_extractor.sampling_rate))

# print the L2 norm of the difference between the original and reconstructed audio
# print(np.linalg.norm(reconstructed_audio - audio_arr) / audio_arr.shape[0])

## 3. Extracting Latent Features from Codes

In [ ]:
codes = encoder_outputs.audio_codes
latent = model.quantizer.decode(codes.transpose(0, 1))

In [ ]:
# bs, n_quantizers, n_frames
batched_codes = torch.concat([codes, codes])
batched_latents = model.quantizer.decode(batched_codes.transpose(0, 1))

In [ ]:
print(codes.shape)
dense_features = model.quantizer.decode(codes)
print(dense_features.shape)

In [ ]:
def save_mimi_features(uid: str, target_dir: str | Path) -> None:
    sample = get_data_sample(fs, uid)
    sem_tokens, ac_tokens = get_mimi_features(
        sample.get_single_channel_audio(rate=feature_extractor.sampling_rate, mask_transcript=True),
        src_rate=feature_extractor.sampling_rate,
        num_quantizers=NUM_QUANTIZERS,
    )

    data = {
        "ac_tokens": ac_tokens,
        "sem_tokens": sem_tokens,
    }

    target_file = Path(target_dir) / f"{uid}.{NUM_QUANTIZERS}q.npz"
    np.savez(target_file, **data)
    return None

In [ ]:
%%capture
save_mimi_features(pair[0], "/home/henry/dev/overfit-duplex/asset/single_pair_dataset")
save_mimi_features(pair[1], "/home/henry/dev/overfit-duplex/asset/single_pair_dataset")